# [Advent of Code 2017](http://adventofcode.com/2017)

In previous years I've done Advent in Code in Go (2015) and Elixir (2016).
In 2016 I compared solutions with [Cameron](https://github.com/wheat779).

He was completing the questions in Python which required I brush up on Python to be able to assist him with improving his solutions as it had been a number of years since I had touched the language.

With the rise of Python in the Data Science and Machine Learning space I've decided to use Python as my language of choice this year. I'm completing my answers in a Notebook to document the thought processes behind my answers.

To warm up for 2017 I've completed a few of the 2016 questions in Python and begun translating a number of general helper functions to Python from my Elixir answers last year as well as some [Project Euler](https://projecteuler.net) helper functions that *may* come in handy.

I've also stolen and modified a few simple functions from [Peter Norvig's AoC 2016 Notebook](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb).

I've placed these helpers in a Notebook named `AoC_Helpers.ipynb` to allow them to be shared between my 2015, 2016 and 2017 Notebooks.

Here, I use the [ipnyb](https://github.com/ipython/ipynb) module to import that Notebook as if it were a normal Python module.

In [1]:
#! pip install attrs

In [2]:
from ipynb.fs.full.aoc_helpers import *

When doing Project Euler I've implemented plenty of graphs and prime sieves so I'll be using more efficient libraries for these in case they come up.

In [3]:
#! pip install networkx primesieve numba
import networkx
import primesieve
from numba import jit

In [4]:
import re
import numpy as np
import math
import itertools as it
import operator

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache, partial,reduce
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

As the Helpers module is shared between 2015, 2016 and 2017, I'm partially applying the `Input` function to specify the year.

`Input(day, year) -> partial(Input, year=2017) -> Input(3) == Input(3, 2017)`

This allows me to use `Input(3)` in the 2016 file to get day 3 of 2016, as well as `Input(3)` in the 2017 file to get day 3 of 2017.

In [5]:
Input = partial(Input, year=2017)

## [Day 11](http://adventofcode.com/2017/day/11): Hex-Ed


[This tutorial](https://www.redblobgames.com/grids/hexagons/) has a great explanation of hex grids. I used the Cube grid as it seems the most simple to use once you get the directions set up.

In [6]:
Hex = namedtuple('Hex', 'x y z')

directions = {
    'n':  Hex( 1,  0, -1),
    's':  Hex(-1,  0,  1),
    'ne': Hex( 0,  1, -1),
    'sw': Hex( 0, -1,  1),
    'se': Hex(-1,  1,  0),
    'nw': Hex( 1, -1,  0),
}

In [7]:
def distance(a, b):
    return max(abs(a.x - b.x), abs(a.y - b.y), abs(a.z - b.z))

def hex_walkabout(instructions, part_b=False):
    start = Hex(0, 0, 0)
    location = start

    max_dist = 0
    for move in instructions:
        m = directions[move]
        location = Hex(location.x + m.x, location.y + m.y, location.z + m.z)
        max_dist = max(max_dist, distance(start, location))

    if part_b: return max_dist
    return distance(start, location)

In [8]:
day11 = Input(11).read().strip().split(',')

hex_walkabout(day11)
hex_walkabout(day11, part_b=True)

1603

# [Day 12](http://adventofcode.com/2017/day/12): Digital Plumber

In [9]:
connections = defaultdict(list)
def parse_12(inp):
    inp = inp.replace('<-> ', '').replace(',', '').strip()
    origin, *destinations = inp.split(' ')
    return (origin, destinations)

for line in Input(12):
    o, ds = parse_12(line)
    for d in ds:
        connections[o].append(d)
        connections[d].append(o)

In [10]:
def get_group(start, seen=set()):
    seen.add(start)
    for con in connections[start]:
        if con in seen: continue
        subgroup = get_group(con, seen)
        seen.update(subgroup)
    return seen

How many nodes connect to 0?

In [11]:
len(get_group('0'))

141

How many total groups are there?

In [12]:
found = set()
groups = 0
for con in connections.keys():
    if con in found: continue
    groups += 1
    for g in connections[con]:
        found.update(get_group(g))
groups

171

## [Day 13](http://www.adventofcode.com/2017/day/13): Packet Scanners

In [13]:
in13 = list(Input(13))

For my first attempt at this for some reason I figured it'd be faster to not look too hard at the problem and just brute force simulating the scanners.

This worked for part A but quickly proved to be inefficient for part B

In [14]:
def setup(inp):
    layers = defaultdict(deque)
    dirs = {}
    for line in inp:
        layer, depth = line.split(': ')
        dirs[int(layer)] = False # false is down true is up
        layers[int(layer)] = deque([True] + [False] * (int(depth) - 1))
    return layers, dirs

In [15]:
def step(layers, dirs):
    for layer, values in layers.items():
        if layers[layer]:
            if not dirs[layer]: values.rotate(1)
            else: values.rotate(-1)
            if dirs[layer] and layers[layer][0]:
                dirs[layer] = False
            elif not dirs[layer] and layers[layer][-1]:
                dirs[layer] = True
    return layers, dirs

In [16]:
layers, dirs = setup(in13)
severity = 0
for i in range(89):
    if layers[i]:
        if layers[i][0]:
            severity += i * len(layers[i])
    step(layers, dirs)
severity

648

In [17]:
def gets_through1(n):
    layers, dirs = setup(Input(13))
    for _ in range(n):
        layers, dirs = step(layers, dirs)
    for i in range(89): # 89
        if layers[i] and layers[i][0]:
            return False
        layers, dirs = step(layers, dirs)

    return True

In [18]:
def delay_needed1():
    for delay in it.count():
        if delay % 1000 == 0: print(delay)
        if gets_through1(delay):
            return delay

I let this run while I watched some TV. 30 minutes later I gave up and determined I needed to do think about it for a few minutes. 10 minutes later, I had my answer!

In order to not actually simulate each step we need to determine the cycle length for each layer.  
```
[1 0 0] 4  
[1 0 0 0] 6  
[1 0 0 0 0] 8  
[1 0 0 0 0 0 0 0 0] 16 == (9 - 1) * 2 == (length - 1) * 2 
```

In [19]:
def parse_13(inp):
    return {layer: depth for layer, depth in map(parse_ints, inp)}

def blocking(position, depth, delay=0):
    cycle_length = (depth - 1) * 2
    return (position + delay) % cycle_length == 0

def count_collisions(layers):
    return sum([position * depth
                for position, depth in layers.items()
                if blocking(position, depth)])

def gets_through(layers, delay):
     return all(not blocking(position, depth, delay)
                for position, depth in layers.items())

def delay_needed(layers):
    for delay in it.count():
        if gets_through(layers, delay):
            return delay

In [20]:
layers = parse_13(in13)

In [21]:
count_collisions(layers)

648

In [22]:
delay_needed(layers)

3933124

## [Day 14](http://www.adventofcode.com/2017/day/14): Disk Defragmentation

In [23]:
def knothash(inp):
    inp = list(map(ord, inp)) + [17, 31, 73, 47, 23]
    rounds = 64
        
    hashes = list(range(256))    
    current_position = 0
    skip = 0
    
    for _ in range(rounds):
        for length in inp:
            if length <= len(hashes):
                hashes = rotate(hashes, current_position)
                hashes[:length] = list(reversed(hashes[:length]))
                hashes = rotate(hashes, -current_position)
            current_position += length + skip
            skip += 1 

    final_hash = ''
    for start in range(0, len(hashes), 16):
        part = hashes[start:start+16]
        hashpart = reduce(operator.xor, part)
        final_hash += f'{hashpart:02x}'
    return final_hash

In [24]:
in14 = Input(14).read().strip()

In [25]:
def hex_to_bin(hex_):
    return "{0:04b}".format(int(hex_, 16))

In [26]:
def to_ascii(bin_):
    return '#' if bin_ == '1' else '.'

In [27]:
rows = [knothash(f'{in14}-{n}') for n in range(128)]
rows = [cat(map(hex_to_bin, row)) for row in rows]
rows = [list(map(to_ascii, row)) for row in rows]

In [28]:
sum(col == '#' for row in rows for col in row)

8216

In [29]:
def floodfill(rows, x, y, num):
    if rows[y][x] != '#': return False
    rows[y][x] = num
    if x > 0:   floodfill(rows, x-1, y, num)
    if x < 127: floodfill(rows, x+1, y, num)        
    if y > 0:   floodfill(rows, x, y-1, num)        
    if y < 127: floodfill(rows, x, y+1, num)
    return True

In [30]:
num = 1
rowb = rows[:]
for y in range(128):
    for x in range(128):
        if floodfill(rowb, x, y, num):
            num += 1
len(set([item for row in rowb for item in row])) - 1 # -1 because of the blank spaces between groups is an element

1139

In [31]:
for row in rowb[:10]:
    print(cat([f'{i:>2}' for i in row[:10]]))

 1 1 . 1 . . . . 2 .
 1 1 1 1 . .30 . . .
 . 1 . 1 1 . . 1 .40
 . . . 1 1 1 1 1 1 .
 .67 . 1 1 . . . . .
 .6767 . 1 . 1 . . 3
83 .67 . 1 1 1 . 3 3
 . . . . 1 1 1 . 3 .
96969696 . . . . . 3
969696 . . 3 3 . 3 3


## [Day 15](http://www.adventofcode.com/2017/day/15): Dueling Generators

In [32]:
A, Af = 722, 16807
B, Bf = 354, 48271

In [33]:
@jit
def generator(gen, factor, modby=1):
    while True:
        gen = (gen * factor) % 2147483647
        if gen % modby == 0:
            yield gen & 0xFFFF

In [34]:
ga = generator(A, Af)
gb = generator(B, Bf)
%time sum(next(ga) == next(gb) for _ in range(40_000_000)) # 612

Wall time: 11.3 s


612

In [35]:
ga = generator(A, Af, 4)
gb = generator(B, Bf, 8)
sum(next(ga) == next(gb) for _ in range(5_000_000)) # 285

285

## [Day 16](http://www.adventofcode.com/2017/day/16): Permutation Promenade

In [36]:
in16 = Input(16).read().strip()

In [37]:
from string import ascii_lowercase
dancers = ascii_lowercase[:16]

In [38]:
def dance(dancers):
    dancers = list(dancers)
    for line in in16.split(','):
        command, args = line[0], line[1:]
        if command == 's': 
            i = int(args)
            dancers = rotate(dancers, -i)
        elif command == 'x':
            a, b = map(int, args.split('/'))
            dancers[a], dancers[b] = dancers[b], dancers[a]
        elif command == 'p':
            a, b = map(dancers.index, args.split('/'))
            dancers[a], dancers[b] = dancers[b], dancers[a]
    return cat(dancers)

For Part B oddly enough I generalized the `find_cycle` method I made for Day 6 so we can use it here to see how long a cycle happens to be in this input.

In [39]:
find_cycle(dancers, dance)

{'cycle_item': 'abcdefghijklmnop',
 'cycle_length': 24,
 'first_seen': 0,
 'total_steps': 24}

So if we take a billion mod the cycle length we can see how many more iterations we need to do to "simulate" 1 billion cycles.

In [40]:
1_000_000_000 % 24

16

In [41]:
def dance_more(dancers, n):
    for i in range(n):
        dancers = dance(dancers)
    return dancers

In [42]:
dance_more(dancers, 16)

'ifocbejpdnklamhg'

## [Day 17](http://www.adventofcode.com/2017/day/17): Spinlock 

In [43]:
def spinlock(goal):
    spinlock = []
    spinlock.append(0)

    pos = 0
    for i in range(1, goal + 1):
        pos = (pos + 370) % len(spinlock)
        spinlock.insert(pos + 1, i)
        pos += 1
    return spinlock[spinlock.index(goal) + 1]

In [44]:
spinlock(2017)

1244

Since we never insert before our current location 0 never moves from index 0. We can just track the last element inserted at index 1.

In [45]:
@jit
def spinlock_second():
    pos, length, goal = 0, 1, 0

    for i in range(1, 50_000_000 + 1):
        pos = (pos + 370) % length + 1
        length += 1
        if pos == 1: goal = i
    return goal

In [46]:
spinlock_second()

11162912

Thanks to numba.jit we can turn this from 10 seconds to a half a second

Hm whoa deque has rotate. Why did I make my own, much slower version of that?

In [47]:
def spinlock2(times, goal=None):
    if goal is None:
        goal = times
        
    spinlock = deque([0])

    for i in range(1, times + 1):
        spinlock.rotate(-370)
        spinlock.append(i)
    
    return spinlock[(spinlock.index(goal) + 1) % len(spinlock)]

In [48]:
#%time spinlock2(50_000_000, 0) # 45 seconds

In [49]:
spinlock2(2017)

1244

Haha holy crap thats fast enough that we can just brute force it. Welp, no more using my homegrown rotate if I need to do so in a tight loop! Thanks, deque!

## [Day 18](http://www.adventofcode.com/2017/day/18): Duet

In [50]:
in18 = list(Input(18))

In [51]:
registers = defaultdict(int)

def val(a):
    if a.lstrip('-').isdigit():
        return int(a)
    return registers[a]

def solvea(in18):
    lastplayed = 0
    inlen = len(in18)
    instruction = 0
    while instruction < inlen:
        c, *args = in18[instruction].strip().split(' ')
        if len(args) == 2:
            a, b = args
        else:
            a = args[0]
        if c == "add":
            registers[a] += val(b)
        elif c == "set":
            registers[a] = val(b)
        elif c == "jgz":
            if val(a) > 0:
                instruction += val(b)
                continue
        elif c == "snd":
            lastplayed = val(a)
        elif c == "mul":
            registers[a] *= val(b)
        elif c == "mod":
            registers[a] %= val(b)
        elif c == "rcv":
            if val(a) > 0:
                return lastplayed
        instruction += 1      

In [52]:
solvea(in18)

4601

In [53]:
def prog(N, inQ, outQ, inp):
    registers = defaultdict(int)
    registers['p'] = N
    inst = 0
    sendcount = 0

    def val(a):
        if a.lstrip('-').isdigit(): return int(a)
        return registers[a]

    while inst < len(inp):
        c, *args = inp[inst].strip().split(' ')
        if len(args) == 2:
            a, b = args
        else:
            a = args[0]

        if   c == "add": registers[a] += val(b)
        elif c == "set": registers[a]  = val(b)
        elif c == "mul": registers[a] *= val(b)
        elif c == "mod": registers[a] %= val(b)
        elif c == "jgz": 
            if val(a) > 0:
                inst += val(b) - 1
        elif c == "snd":
            outQ.append(val(a))
            sendcount += 1
        elif c == "rcv":
            if len(inQ) > 0:
                registers[a] = inQ.popleft()
            else:
                inst -= 1
                # Since we're blocked on our queue, return False.
                yield (False, sendcount)
        # If we're not done iterating or held up, return True.
        inst += 1
        yield (True, sendcount)
    # Since we've gone through all the instructions, return False.
    yield (False, sendcount)

In [54]:
def solve18b():
    Q0 = deque()
    Q1 = deque()
    prog0 = prog(0, Q0, Q1, in18)
    prog1 = prog(1, Q1, Q0, in18)

    while True:
        more_0, sendcount_0 = next(prog0)
        more_1, sendcount_1 = next(prog1)
        if not more_0 and not more_1:
            break

    return sendcount_1

In [55]:
solve18b()

6858

## [Day 19](http://www.adventofcode.com/2017/day/19): A Series of Tubes

In [56]:
from string import ascii_uppercase

In [57]:
dirs = {'up':   Point(0,-1),  'down':  Point(0, 1),
        'left': Point(-1, 0), 'right': Point(1, 0)}
line, corner, empty, letters = '-|', '+', ' ', ascii_uppercase

def new_direction(dir, pos):
    if dir in ('down', 'up'):    possible_dirs = ('left', 'right')
    if dir in ('left', 'right'): possible_dirs = ('up', 'down')

    for direction in possible_dirs:
        npos = pos + dirs[direction]
        what = diagram[npos.y][npos.x]
        if what is not empty:
            return direction
    return False

def find_entrance(diagram): return Point(diagram[0].index('|'), 0)
        
def follow_diagram(diagram):
    dir = 'down'
    pos = find_entrance(diagram)
    what = diagram[pos.y][pos.x]
    found, steps = '', 0
    
    while what not in empty:
        if   what in letters: found += what
        elif what in corner:  dir = new_direction(dir, pos)
        steps += 1
        pos = pos + dirs[dir]
        what = diagram[pos.y][pos.x]

    return found, steps

In [58]:
diagram = list(map(lambda x: x.strip('\n'), Input(19)))

follow_diagram(diagram)

('BPDKCZWHGT', 17728)

## [Day 20](http://www.adventofcode.com/2017/day/20): Particle Swarm

In [59]:
in20 = list(Input(20))

First, lets define a few classes to hold this information to keep our code tidier later.

In [60]:
@attr.s(hash=True)
class XYZ():
    x = attr.ib()
    y = attr.ib()
    z = attr.ib()
    
    def __add__(self, other):
        return XYZ(self.x + other.x, self.y + other.y, self.z + other.z)
        
    def __iadd__(self, other):
        self.x += other.x
        self.y += other.y
        self.z += other.z
        return self

@attr.s
class Particle():
    number = attr.ib()
    position = attr.ib()
    velocity = attr.ib()
    acceleration = attr.ib()
        
    def step(self):
        self.velocity += self.acceleration
        self.position += self.velocity
    
    def distance(self):
        return abs(self.position.x) + abs(self.position.y) + abs(self.position.z)

With that we just need to parse the input into a Particle!

In [61]:
def parsicles(inp):
    particles = {}
    for n, line in enumerate(inp):
        p, v, a = [XYZ(*parse_ints(s)) for s in line.split(', ')]
        particles[n] = Particle(n, p, v, a)
    return particles

### Part A
For A we need to find the particle which will end up the closest to (0, 0, 0) in our simulation.

In [62]:
def closest_particle():
    particles = parsicles(in20)
    for _ in range(400):
        for p in particles.values():
            p.step()
    return min(particles.values(), key=lambda p: p.distance())

In [63]:
closest_particle()

Particle(number=119, position=XYZ(x=106884, y=-751, z=49865), velocity=XYZ(x=472, y=-2, z=320), acceleration=XYZ(x=1, y=0, z=1))

### Part B
For B any particles in the same space at a given step in the simulation collide and are removed! How many particles do we end up with?

In [64]:
def particle_fightclub():
    particles = parsicles(in20)
    for _ in range(400):
        seen = defaultdict(list)
        for n, p in particles.items():
            seen[p.position].append(n)
            
        for parts in seen.values():
            if len(parts) > 1:
                for n in parts:
                    del particles[n]
                    
        for p in particles.values():
            p.step()
            
    return len(particles)

In [65]:
particle_fightclub()

471